In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from pathlib import Path
import pandas as pd
import pickle
import numpy as np
# from tqdm import tqdm_notebook as tqdm

import torch.utils.data
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator, Engine
from ignite.metrics import Accuracy, Loss

from src.models import InsiderClassifier, LSTM_Encoder
from src.params import get_params
from src.dataset import CertDataset, create_data_loaders
from src.cnn_trainer import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# output_dir = Path(r'C:\Users\Mvideo\Google Drive\Datasets\CERT_output')
# answers_dir = Path(r"C:/Users/Mvideo/Downloads/answers")

output_dir = Path(r'C:\Users\admin\Google Drive\Datasets\CERT_output')
answers_dir = Path(r"C:\Users\admin\Google Drive\Datasets\CERT\answers")

lstm_checkpoint = output_dir / 'checkpoints/final3-nll/final.pkl'

run_name = 'cnn-test'

main_answers_file = answers_dir / "insiders.csv"

In [5]:
actions, targets = CertDataset.prepare_dataset(output_dir / 'aggregated.pkl', main_answers_file, min_length=50, max_length=200)

C:\Anaconda3\envs\torch\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [6]:
cert_dataset = CertDataset(actions, targets)
train_loader, val_loader = create_data_loaders(cert_dataset, validation_split=0.3, random_seed=0, batch_size=4)

params = get_params()

device = 'cuda'

In [ ]:
model = InsiderClassifier(params['model'], lstm_checkpoint)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

train_engine = create_supervised_trainer(
                                        model, optimizer, criterion, device=device,
                                        prepare_batch=prepare_batch,
                                        log_dir=(output_dir / 'logs' / 'cnn' / run_name).as_posix(),
                                        checkpoint_dir=output_dir / 'checkpoints' / 'cnn' / run_name
                                       )

In [11]:
train_engine.run(train_loader, max_epochs=500)

Current run is terminating due to exception: .
Engine run is terminating due to exception: .


KeyboardInterrupt: 

In [10]:
%debug

> c:\users\mvideo\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py(2044)binary_cross_entropy()
   2042     if input.numel() != target.numel():
   2043         raise ValueError("Target and input must have the same number of elements. target nelement ({}) "
-> 2044                          "!= input nelement ({})".format(target.numel(), input.numel()))
   2045 
   2046     if weight is not None:



ipdb>  u


> c:\users\mvideo\appdata\local\programs\python\python37\lib\site-packages\torch\nn\modules\loss.py(498)forward()
    496 
    497     def forward(self, input, target):
--> 498         return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
    499 
    500 



ipdb>  u


> c:\users\mvideo\appdata\local\programs\python\python37\lib\site-packages\torch\nn\modules\module.py(547)__call__()
    545             result = self._slow_forward(*input, **kwargs)
    546         else:
--> 547             result = self.forward(*input, **kwargs)
    548         for hook in self._forward_hooks.values():
    549             hook_result = hook(self, input, result)



...

ipdb>  criterion(scores[:,0], target.float()


*** SyntaxError: unexpected EOF while parsing


ipdb>  criterion(scores[:,0], target.float())


tensor(0.7209, grad_fn=<BinaryCrossEntropyBackward>)


ipdb>  exit


In [ ]:
val_engine = create_supervised_evaluator(
        model, device=device,
        prepare_batch=prepare_batch,
        metrics={},
        criterion=criterion,
        log_dir=(output_dir / 'logs' / run_name).as_posix(),
)

@train_engine.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    print('Validation run:')
    val_engine.run(val_loader)

In [159]:
%debug

> c:\users\mvideo\dropbox\edu\semestr_m2\cert\src\models.py(61)forward()
     59 
     60         def forward(self, x):
---> 61                 assert(x.shape[1] == self.seq_length)
     62                 assert(x.shape[2] == self.lstm_hidden_size)
     63 



ipdb>  x.shape


torch.Size([4, 1, 200, 40])


ipdb>  exit
